# **Dataset Creation**
### Notebook for the testing and development of a methodology for creating a fully labelled dataset of molecules and their functional groups.

In [1]:
import rdkit
import rdkit.Chem as Chem
import efgs
from joblib import Parallel, delayed
import pandas as pd
from rdkit.Chem import AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
import pickle
import os
import math



## Dataset Creation
#### Starting with the small Pec50 data

### Compute the efgs in parallel

In [2]:
PATT=Chem.MolFromSmarts("[$([D1]=[*])]")
REPL=Chem.MolFromSmarts("[*]")

def get_scaffold(mol,real_bm=True,use_csk=False,use_bajorath=False):
    Chem.RemoveStereochemistry(mol) #important for canonization of CSK!
    scaff=MurckoScaffold.GetScaffoldForMol(mol)
    if use_bajorath:
        scaff=AllChem.DeleteSubstructs(scaff, PATT)
    if real_bm:
        scaff=AllChem.ReplaceSubstructs(scaff,PATT,REPL,replaceAll=True)[0]                                          
    if use_csk:
        scaff=MurckoScaffold.MakeScaffoldGeneric(scaff)
        if real_bm:
            scaff=MurckoScaffold.GetScaffoldForMol(scaff)
    return scaff


## Chembl Dataset

In [3]:
# Load the chembl data
chembl_data = pd.read_csv('chembl_35_cleaned.csv',header=None, names=['smiles'])

In [4]:
def safe_mol_from_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol
    except:
        return None
    
# 1. Convert SMILES to RDKit Mol
chembl_data['rdkit_mol'] = chembl_data['smiles'].apply(safe_mol_from_smiles)
print('Converted SMILES to RDKit Mol objects for ChEMBL dataset.')


# 2. Generate InChIKeys
chembl_data['inchikey'] = chembl_data['rdkit_mol'].apply(
    lambda mol: Chem.MolToInchiKey(mol) if mol is not None else None
)
print('Generated InChIKeys for ChEMBL dataset.')


[10:55:13] Explicit valence for atom # 7 P, 7, is greater than permitted
[10:57:07] Explicit valence for atom # 10 Si, 6, is greater than permitted
[10:57:36] Explicit valence for atom # 1 P, 7, is greater than permitted
[10:58:45] Explicit valence for atom # 1 As, 7, is greater than permitted


Converted SMILES to RDKit Mol objects for ChEMBL dataset.
Generated InChIKeys for ChEMBL dataset.


In [5]:
# Drop rows with None values in 'rdkit_mol' or 'inchikey'
chembl_data = chembl_data.dropna(subset=['rdkit_mol', 'inchikey'])

In [6]:
def compute_efgs(mol):
    _,_, psmis, _ = efgs.get_dec_fgs(mol)
    return psmis

def compute_efgs_safe(mol):
    try:
        return compute_efgs(mol)
    except Exception as e:
        print(f"Error computing EFGs for molecule: {mol}. Error: {e}")
        return None  # Tuple to match expected unpacking

def compute_scaffold_safe(mol):
    try:
        return get_scaffold(mol)
    except Exception as e:
        return None  # Return None if there's an error

In [7]:
# Configuration
chunk_size = 100000
output_dir = "chembl_chunks"
os.makedirs(output_dir, exist_ok=True)

In [8]:
total_mols = len(chembl_data)
num_chunks = math.ceil(total_mols / chunk_size)

# Function to save each chunk
def save_chunk_results(dir, chunk_idx, results):
    with open(os.path.join(dir, f"chunk_{chunk_idx}.pkl"), "wb") as f:
        pickle.dump(results, f)

In [10]:
# Process in chunks
for chunk_idx in range(num_chunks):
    start = chunk_idx * chunk_size
    end = min((chunk_idx + 1) * chunk_size, total_mols)
    
    print(f"Processing chunk {chunk_idx + 1}/{num_chunks} ({start}:{end})...")
    mols_chunk = chembl_data['rdkit_mol'].iloc[start:end]

    # Only compute if the file doesn't already exist (for resumability)
    chunk_file = os.path.join(output_dir, f"chunk_{chunk_idx}.pkl")
    if not os.path.exists(chunk_file):
        chunk_results = Parallel(n_jobs=-1)(
            delayed(compute_efgs_safe)(mol) for mol in mols_chunk
        )
        save_chunk_results(output_dir, chunk_idx, chunk_results)
    else:
        print(f"Chunk {chunk_idx + 1} already computed, skipping.")

# Recombine results
all_results = []
for chunk_idx in range(num_chunks):
    with open(os.path.join(output_dir, f"chunk_{chunk_idx}.pkl"), "rb") as f:
        chunk_results = pickle.load(f)
        all_results.extend(chunk_results)

# Store or attach results
chembl_data['psmis'] = all_results
print('All PSMIs added to ChEMBL dataset.')

Processing chunk 1/24 (0:100000)...
Chunk 1 already computed, skipping.
Processing chunk 2/24 (100000:200000)...
Chunk 2 already computed, skipping.
Processing chunk 3/24 (200000:300000)...
Chunk 3 already computed, skipping.
Processing chunk 4/24 (300000:400000)...
Chunk 4 already computed, skipping.
Processing chunk 5/24 (400000:500000)...
Chunk 5 already computed, skipping.
Processing chunk 6/24 (500000:600000)...
Chunk 6 already computed, skipping.
Processing chunk 7/24 (600000:700000)...
Chunk 7 already computed, skipping.
Processing chunk 8/24 (700000:800000)...
Chunk 8 already computed, skipping.
Processing chunk 9/24 (800000:900000)...
Chunk 9 already computed, skipping.
Processing chunk 10/24 (900000:1000000)...
Chunk 10 already computed, skipping.
Processing chunk 11/24 (1000000:1100000)...
Chunk 11 already computed, skipping.
Processing chunk 12/24 (1100000:1200000)...
Chunk 12 already computed, skipping.
Processing chunk 13/24 (1200000:1300000)...
Chunk 13 already computed,

In [11]:
print(all_results[0])  # Print the first result to verify

['O=C1C(=N[N]([R])[R])C=N[N]1[R]', '[OH][Car]', '[Nar]', '[Nar]', '[Nar]', '[Nar]']


In [12]:
# Config
chunk_size = 100000
scaffold_output_dir = "chembl_scaffold_chunks"
os.makedirs(scaffold_output_dir, exist_ok=True)

total_mols = len(chembl_data)
num_chunks = math.ceil(total_mols / chunk_size)

In [13]:
# Chunked scaffold computation
for chunk_idx in range(num_chunks):
    start = chunk_idx * chunk_size
    end = min((chunk_idx + 1) * chunk_size, total_mols)

    print(f"Processing scaffold chunk {chunk_idx + 1}/{num_chunks} ({start}:{end})...")
    mols_chunk = chembl_data['rdkit_mol'].iloc[start:end]

    chunk_file = os.path.join(scaffold_output_dir, f"chunk_{chunk_idx}.pkl")
    if not os.path.exists(chunk_file):
        # Compute scaffold mols
        scaffold_mols = Parallel(n_jobs=-1)(
            delayed(compute_scaffold_safe)(mol) for mol in mols_chunk
        )
        # Convert to SMILES
        scaffold_smiles = [
            Chem.MolToSmiles(mol) if mol else None for mol in scaffold_mols
        ]
        save_chunk_results(scaffold_output_dir, chunk_idx, scaffold_smiles)
    else:
        print(f"Scaffold chunk {chunk_idx + 1} already computed, skipping.")

# Recombine scaffold SMILES
all_scaffold_smiles = []
for chunk_idx in range(num_chunks):
    with open(os.path.join(scaffold_output_dir, f"chunk_{chunk_idx}.pkl"), "rb") as f:
        scaffold_smiles = pickle.load(f)
        all_scaffold_smiles.extend(scaffold_smiles)

# Store in DataFrame
chembl_data['scaffolds'] = all_scaffold_smiles
print('All scaffolds (as SMILES) added to ChEMBL dataset.')


Processing scaffold chunk 1/24 (0:100000)...
Scaffold chunk 1 already computed, skipping.
Processing scaffold chunk 2/24 (100000:200000)...
Scaffold chunk 2 already computed, skipping.
Processing scaffold chunk 3/24 (200000:300000)...
Scaffold chunk 3 already computed, skipping.
Processing scaffold chunk 4/24 (300000:400000)...
Scaffold chunk 4 already computed, skipping.
Processing scaffold chunk 5/24 (400000:500000)...
Scaffold chunk 5 already computed, skipping.
Processing scaffold chunk 6/24 (500000:600000)...
Scaffold chunk 6 already computed, skipping.
Processing scaffold chunk 7/24 (600000:700000)...
Scaffold chunk 7 already computed, skipping.
Processing scaffold chunk 8/24 (700000:800000)...
Scaffold chunk 8 already computed, skipping.
Processing scaffold chunk 9/24 (800000:900000)...
Scaffold chunk 9 already computed, skipping.
Processing scaffold chunk 10/24 (900000:1000000)...
Scaffold chunk 10 already computed, skipping.
Processing scaffold chunk 11/24 (1000000:1100000)...

In [14]:
chembl_data_final = chembl_data[['smiles', 'inchikey', 'psmis', 'scaffolds']]
chembl_data_final.rename(columns={'psmis': 'fgs'}, inplace=True)
chembl_data_final.to_csv('chembl_35_fg_scaf.csv', index=False)

/var/folders/r4/3wxjq8t93d76lfkb5vmhrlpr0000gn/T/ipykernel_44129/3216569144.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chembl_data_final.rename(columns={'psmis': 'fgs'}, inplace=True)


In [15]:
chembl_data.head

<bound method NDFrame.head of                                                     smiles  \
0        CC1=NN(c2ccc(C)c(C)c2)C(=O)/C1=N\Nc1cccc(-c2cc...   
1                          COCC1c2c(cnn2C)CCN1Cc1ccc(C)cc1   
2           COc1cccc(N(C)C(=O)c2ccc(-c3ccc(OC)c(F)c3)s2)c1   
3            O=c1[nH]c2ccc(-c3csc(-c4ccc(F)c(F)c4)n3)cc2o1   
4              Cc1cc(CC#N)cc(C)c1Nc1ccnc(Nc2ccc(C#N)cc2)n1   
...                                                    ...   
2310846              CC(C)S(=O)(=O)[C@@H](C)C(=O)NCc1ccco1   
2310847     COc1ccc(CN2C[C@H]3CN(C(=O)c4ccno4)CCN3C2=O)cc1   
2310848          CCN(CC(=O)NCC1CCCO1)S(=O)(=O)c1ccc(Cl)cc1   
2310849  CC(C)(C)C(=O)N(C(=O)C(C)(C)C)c1c(C(N)=O)nnn1Cc...   
2310850                    N#Cc1c[nH]c2c(OCc3ccccc3)cccc12   

                                             rdkit_mol  \
0        <rdkit.Chem.rdchem.Mol object at 0x316afa2d0>   
1        <rdkit.Chem.rdchem.Mol object at 0x316afa5e0>   
2        <rdkit.Chem.rdchem.Mol object at 0x316afa3

Exception ignored in: <function ResourceTracker.__del__ at 0x10305c2c0>
Traceback (most recent call last):
  File "/Users/damianelkins/miniconda3/envs/rdkit-thesis/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Users/damianelkins/miniconda3/envs/rdkit-thesis/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Users/damianelkins/miniconda3/envs/rdkit-thesis/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1032c02c0>
Traceback (most recent call last):
  File "/Users/damianelkins/miniconda3/envs/rdkit-thesis/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Users/damianelkins/miniconda3/envs/rdkit-thesis/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Users/damianelkins/miniconda3/envs/rdkit-thesis/lib/python3.13/multiprocessi